In [26]:
from openai import OpenAI
import os
import json
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

In [93]:
def generate_leetcode_question(difficulty: str, data_structures: list, programming_language: str):
    data_structures_str = ", ".join(data_structures)
    client = OpenAI()
    json_schema = {
        "challenge": "",
        "entry_function": "",
        "test_cases": [],
        "solution": ""
    }
    json_schema_str = json.dumps(json_schema, indent=2)
    prompt = f"""As an intelligent Leetcode Builder API, given the following criteria:
- Difficulty: {difficulty}
- Data Structures: {data_structures_str}
- Programming Language: {programming_language}

Create a leetcode question, make it a random scenario where the solution should involve the data structure for the best solution.

The response as follows:
1: "challenge": "text with the Leetcode challenge, give test cases and example result of test case and explain why"
2: "entry_function": "example in Python, e.g., def solve(arr: list) -> int: return -1"
3: "test_cases": "JSON format with input values and output values"
4: "solution": "one way to solve the challenge"

Your task is to generate a response that conforms to the following JSON schema:
{json_schema_str}

Please fill in the schema with appropriate values based on the criteria. Ensure the response strictly adheres to the given schema format."""

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": prompt},
        ],
        temperature=0,
    )
    return response
# Example usage
difficulty = "Medium"
data_structures = ["Array"]
programming_language = "python"
generated_text = generate_leetcode_question(difficulty, data_structures, programming_language)
leetcode = generated_text.copy()

In [94]:
leetcode_dict = json.loads(leetcode.choices[0].message.dict()['content'])
print(leetcode_dict['challenge'])
print(leetcode_dict['entry_function'])
print(leetcode_dict['test_cases'])
print(leetcode_dict['solution'])

Given an array of integers, return the length of the longest increasing subsequence. A subsequence is a sequence that can be derived from another sequence by deleting some or no elements without changing the order of the remaining elements. For example, [3, 6, 2, 7] is a subsequence of the array [0, 3, 1, 6, 2, 2, 7].

Example:
Input: [10, 9, 2, 5, 3, 7, 101, 18]
Output: 4 
Explanation: The longest increasing subsequence is [2, 3, 7, 101], therefore the length is 4. Note that there may be more than one LIS combination, it is only necessary for you to return the length.

Your algorithm should run in O(n^2) complexity. Follow up: Could you improve it to O(n log n) time complexity?
def lengthOfLIS(nums: list) -> int: return -1
[{'input': [10, 9, 2, 5, 3, 7, 101, 18], 'output': 4}, {'input': [0, 1, 0, 3, 2, 3], 'output': 4}, {'input': [7, 7, 7, 7, 7, 7, 7], 'output': 1}]
def lengthOfLIS(nums):
    if not nums:
        return 0

    dp = [1] * len(nums)
    for i in range (1 , len(nums)):
 

In [96]:
import threading    
def compile_and_run_function(code, test_cases):
    local_vars = {}
    exec(code, globals(), local_vars)
    function_name = list(local_vars.keys())[0]
    function = local_vars[function_name]
    def run_test_cases():
        for i, test in enumerate(test_cases):
            input_val = test["input"]
            expected_output = test["output"]
            params = []
            if (type(input_val) == dict):
                input_val_keys = input_val.keys()
                for key in input_val.keys():
                    params.append(input_val[key])
            else:
                params.append(input_val)
            try:
                actual_output = function(*params)
                assert actual_output == expected_output, f"Test #{i+1} failed: {input_val} -> {actual_output} != {expected_output}"
                print(f"Test #{i+1} passed: {input_val} -> {actual_output} == {expected_output}")
            except AssertionError as e:
                print(e)
            except Exception as e:
                print(f"Test #{i+1} raised an unexpected error: {e}")
    test_thread = threading.Thread(target=run_test_cases)
    test_thread.start()
    test_thread.join()
    
entry_function_code = leetcode_dict["entry_function"]
test_cases = leetcode_dict["test_cases"]
compile_and_run_function(entry_function_code, test_cases)

Test #1 passed: [10, 9, 2, 5, 3, 7, 101, 18] -> 4 == 4
Test #2 passed: [0, 1, 0, 3, 2, 3] -> 4 == 4
Test #3 passed: [7, 7, 7, 7, 7, 7, 7] -> 1 == 1
